# Variational Quantum Error Correction with QVECTOR

In this notebook, we will optimize a Quantum Error Correction (QEC) problem. 
Here, our set of states $\{\psi_i\}_{i=1}^N$ are sampled from the Haar distribution (or an approximation thereof) and we optimize the cost 
$$L(\vec{p}, \vec{q}) = \sum_{\vert \psi_i \rangle \in \mathcal{S}} \frac{1}{\vert\mathcal{S}\vert} E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q})$$
where $\vec{p}, \vec{q}$ are parameters of a variational circuit, and $E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q}) = \langle \psi_i \vert \mathcal{V}(\vec{p})^\dagger \mathcal{W}(\vec{q}) \mathcal{V}(\vec{p}) \vert \psi_i \rangle$, with $\mathcal{V}$ the encoding operator and $\mathcal{W}$ the decoding operator.

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

# Initialise the number of qubits that interest us.
k, n, r = 1, 5, 2
nbqbits = n + r

# Build the dataset here with qml.SimplifiedTwoDesign
# See https://docs.pennylane.ai/en/stable/code/api/pennylane.SimplifiedTwoDesign.html
dataset = None # TODO

In [ ]:
# Get the Hamiltonian of interest
coefficients_cost = [- 1.0 / k for _ in range(k)]
projector = np.zeros((2, 2))
projector[0, 0] = 1
vqec_hamiltonian_term = [qml.Hermitian(projector,wires=i) for i in range(k)]
hermitian_of_interest = qml.Hamiltonian(coefficients_cost, vqec_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [ ]:
analytic_dev = qml.device("default.qubit", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint):
    # We might have to change the weights shape here to differentiate between p and q
    
    # Apply S         to range(k)     to prepare an approximate 2-design state
    # Apply V(p)      to range(n)     to encode the state
    # Apply W(q)      to range(n + r) to correct potential errors
    # Apply V^{-1}(p) to range(n)     to decode the state
    # Apply S^{-1}    to range(k)     to un-prepare.
    
    # Measure the probability to get the all-zero |0...0> output.
    return qml.expval(hamiltonian_of_interest)

def cost_analytic_alldataset(weights):
    
    cost = 0.0
    for m in range(nbdatapoints):
        cost += cost_analytic_one_circuit(weights, m)
    cost = 1.0 + cost / nbdatapoints
    return cost

Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

In [ ]:
from pennylane.templates.layers import StronglyEntanglingLayers

# hyperparameter of ansatz
num_layers = 3

param_shape = StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=nbqbits)
init_params = np.random.uniform(low=0.0, high=2*np.pi, size=param_shape, requires_grad=True)
cost_analytic_alldataset(init_params)

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [ ]:
opt = Refoqus(moldataset, vqse_hamiltonian_term, coefficients_cost, param_shape, min_shots=2)
params = init_params
niter = 20

cost_refoqus = [cost_analytic_alldataset(params)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")